### Using Big Query

Import the python package:

In [1]:
# ! pip install google.cloud.bigquery

In [2]:
import os

from google.cloud import bigquery

The first sptep in this workflow is to create a Client object to connect on the bigquery datastore

In [3]:
# Define a variavel de ambiente do caminho da chave de conexão

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"C:\Users\felip\Desktop\References\Scripts\notebooks\big_query_key.json"

In [4]:
client = bigquery.Client()

In [5]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

In [6]:
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the list_tables() method to list the tables in the dataset.

In [7]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

for table in tables:
    print(table.table_id)

comments
full
full_201510
stories


Similar to how we fetched a dataset, we can fetch a table. In the code cell below, we fetch the full table in the hacker_news dataset.

In [8]:
# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

### Table schema

The structure of a table is called its schema. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the full table that we fetched above.

In [9]:
# Print information on all the columns in the "full" table in the "hacker_news" dataset
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', 'Type of details (comment, comment_ranking, poll, story, job, pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', 'Number of story or poll descendants', (), None),
 SchemaField

the first SchemaField (name, fieldtype or datatype, mode, description) tell us that the column is called 'title', the datatype id 'string', Null values are allowed, contains the story title  

the method list_rows show the first rows to make sure this right, sometimes databases have outdate descriptions

In [15]:
client.list_rows(table, max_results=5).to_dataframe()

C:\Users\felip\Miniconda3\lib\site-packages\google\cloud\bigquery\table.py:1763: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,I used to do the same but in analog way when I...,None,victornomad,NaN,1516616659,2018-01-22 10:24:19+00:00,comment,16203317,16199871,NaN,NaN,None
1,None,None,It&#x27;s very interesting to take apart some ...,None,cup-of-tea,NaN,1516616657,2018-01-22 10:24:17+00:00,comment,16203316,16203127,NaN,NaN,None
2,None,None,"I&#x27;ve seen kids run, doesn&#x27;t mean it&...",None,maaaats,NaN,1516616645,2018-01-22 10:24:05+00:00,comment,16203315,16202422,NaN,NaN,None
3,None,None,"Apologies. I should have been more clearer, to...",None,pan69,NaN,1516616640,2018-01-22 10:24:00+00:00,comment,16203314,16203171,NaN,NaN,None
4,None,None,"Yes, we do. The ecosystem is great and microse...",None,ivan_gammel,NaN,1516616616,2018-01-22 10:23:36+00:00,comment,16203310,16203084,NaN,NaN,None


In [16]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:1], max_results=5).to_dataframe()

C:\Users\felip\Miniconda3\lib\site-packages\google\cloud\bigquery\table.py:1763: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,title
0,None
1,None
2,None
3,None
4,None


## Disclaimer

Cada usuario do kaggle pode usar 5TB a cada 30 dias (nos exercicios). Apos isso tera que esperar até poder usar novamente

### Bloco 2 - SELECT, FROM & WHERE

In [20]:
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

dataset = client.get_dataset(dataset_ref)

In [29]:
for table in list(client.list_tables(dataset)):
    print(table.table_id)

global_air_quality


In [31]:
table_ref = dataset_ref.table("global_air_quality")

table = client.get_table(table_ref)

In [33]:
client.list_rows(table, max_results=5).to_dataframe()

C:\Users\felip\Miniconda3\lib\site-packages\google\cloud\bigquery\table.py:1763: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the REST endpoint.
  if not self._validate_bqstorage(bqstorage_client, create_bqstorage_client):


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25


In [46]:
table.schema

[SchemaField('location', 'STRING', 'NULLABLE', 'Location where data was measured', (), None),
 SchemaField('city', 'STRING', 'NULLABLE', 'City containing location', (), None),
 SchemaField('country', 'STRING', 'NULLABLE', 'Country containing measurement in 2 letter ISO code', (), None),
 SchemaField('pollutant', 'STRING', 'NULLABLE', 'Name of the Pollutant being measured. Allowed values: PM25, PM10, SO2, NO2, O3, CO, BC', (), None),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Latest measured value for the pollutant', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', 'The datetime at which the pollutant was measured, in ISO 8601 format', (), None),
 SchemaField('unit', 'STRING', 'NULLABLE', 'The unit the value was measured in coded by UCUM Code', (), None),
 SchemaField('source_name', 'STRING', 'NULLABLE', 'Name of the source of the data', (), None),
 SchemaField('latitude', 'FLOAT', 'NULLABLE', 'Latitude in decimal degrees. Precision >3 decimal points.', (), None),
 SchemaF

Everything looks good! So, let's put together a query. Say we want to select all the values from the city column that are in rows where the country column is 'US' (for "United States").

In [38]:
query="""
    SELECT city
    FROM `bigquery-public-data.openaq.global_air_quality`
    WHERE country = 'US'
"""

Submitting the query to the dataset

In [39]:
# Creating a client object
client = bigquery.Client()

In [40]:
# Set up the query
query_job = client.query(query)

Next, we run the query and convert the results to a pandas DataFrame.

In [41]:
# API request - run the query, and return a pandas Dataframe
us_cities = query_job.to_dataframe()

C:\Users\felip\Miniconda3\lib\site-packages\google\cloud\bigquery\client.py:448: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


Now we've got a pandas DataFrame called us_cities, which we can use like any other DataFrame.

In [44]:
us_cities.head()

,city
0,BROWN
1,BROWN
2,BROWN
3,BROWN
4,Houston


In [45]:
us_cities.shape

(3723, 1)

In [43]:
# What five cities have the most measurements?
us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
New York-Northern New Jersey-Long Island    60
Riverside-San Bernardino-Ontario            60
Name: city, dtype: int64

try to select more than one row

In [58]:
query = """
SELECT city, country
FROM `bigquery-public-data.openaq.global_air_quality`
WHERE country = 'US'
"""

In [59]:
query_job = client.query(query)

In [85]:
df = query_job.to_dataframe()

df.head(5)

C:\Users\felip\Miniconda3\lib\site-packages\google\cloud\bigquery\client.py:448: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,city,country
0,BROWN,US
1,BROWN,US
2,BROWN,US
3,BROWN,US
4,Houston,US


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3723 entries, 0 to 3722
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city     3723 non-null   object
 1   country  3723 non-null   object
dtypes: object(2)
memory usage: 58.3+ KB


In [88]:
df.city.value_counts()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
New York-Northern New Jersey-Long Island    60
Riverside-San Bernardino-Ontario            60
                                            ..
Kalispell                                    1
MINERAL                                      1
Barnstable Town                              1
RACINE                                       1
Sioux City                                   1
Name: city, Length: 781, dtype: int64

### Working with big datasets

we'll teach you how to avoid scanning too much data at once, so that you don't run over your limit.

To begin,**you can estimate the size of any query before running it**. Here is an example using the (very large!) Hacker News dataset. To see how much data a query will scan, we create a QueryJobConfig object and set the dry_run parameter to True.

In [89]:
# Query to get the score column from every row where the type column has value "job"

In [90]:
query = """
    SELECT score, title
    FROM `bigquery-public-data.hacker_news.full`
    WHERE type = "job" 
"""

In [91]:
# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes".format(dry_run_query_job.total_bytes_processed))

You can also specify a parameter when running the query to limit how much data are you willing to scan. Here a example:

In [96]:
# Only run the query if it's less than 1 MB

ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# set up the query (will only run if it's les than 1MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return pandas DataFrame
safe_query_job.to_dataframe()

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/optimal-vial-303201/queries/561043aa-10a2-4fa5-b154-eb06cda21c88?maxResults=0&location=US&prettyPrint=false: Query exceeded limit for bytes billed: 1000000. 448790528 or higher required.

(job ID: 561043aa-10a2-4fa5-b154-eb06cda21c88)

           -----Query Job SQL Follows-----           

    |    .    |    .    |    .    |    .    |
   1:
   2:    SELECT score, title
   3:    FROM `bigquery-public-data.hacker_news.full`
   4:    WHERE type = "job" 
    |    .    |    .    |    .    |    .    |

In this case, the query was cancelled, because the limit of 1 MB was exceeded. However, we can increase the limit to run the query successfully!

In [103]:
# Only run the query if it's less than 1GB
ONE_GB =1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# set up the query (will only run if it's less the 1GB)
safe_query_job = client.query(query, job_config=safe_config)

# API resques - try to run the query and return a pandas DataFrame
df = safe_query_job.to_dataframe()

df.score.mean()

1.8506144393241168